In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### 1. Answer Questions 1 through 5 of part 6 of the classwork. You can work right in the notebook from class if you wish, or create a new notebook (or paste it below). Please copy and paste each question into its own cell with answers below it.

In [ ]:
l = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)', header=0)
world_gdp = l[1].iloc[3:194,1:3].reset_index(drop=True)
world_gdp.columns = ['Country', 'GDP']
world_gdp['Country'] = world_gdp['Country'].str.split('[').str[0]
world_gdp['GDP'] = world_gdp['GDP'].str.split('/').str[0].str.replace(',','')
world_gdp['GDP'] = world_gdp['GDP'].astype('float')
world_gdp['GDP_Log'] = np.log(world_gdp['GDP'])

#### 1. Plot the histogram, swarmplot, boxplot, and ECDF of the GDP Log. How does this histogram differ from the histogram we saw of the median age above? How does the ECDF differ from the ECDF we saw of the median age? What else can we tell about the distribution from these plots? Are there any outliers?

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.hist(world_gdp['GDP_Log'])

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.swarmplot(data=world_gdp['GDP_Log'])

In [ ]:
fig = plt.figure(figsize=(10,10))
n,bins,patches = plt.hist(world_gdp['GDP_Log'], cumulative=True, bins=50, density=True, histtype='step');
#This extra line removes the final line from the graph
patches[0].set_xy(patches[0].get_xy()[:-1])

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.boxplot(world_gdp['GDP_Log'])

This histogram has a normal distribution, while the histogram we looked at during class has more of a uniform distribution. This is also seen in the ECDF, which is curved compared to the relatively straight ECDF of the countries' ages. There is one outlier on the lower end.

#### 2. Find the mean, median, variance, standard deviation, maximum, minimum, range, and Interquartile Range of the GDP Log.

In [ ]:
np.mean(world_gdp['GDP_Log'])

In [ ]:
np.median(world_gdp['GDP_Log'])

In [ ]:
np.var(world_gdp['GDP_Log'])

In [ ]:
np.std(world_gdp['GDP_Log'])

In [ ]:
np.max(world_gdp['GDP_Log'])

In [ ]:
np.min(world_gdp['GDP_Log'])

In [ ]:
np.max(world_gdp['GDP_Log']) - np.min(world_gdp['GDP_Log'])

In [ ]:
np.percentile(world_gdp['GDP_Log'], 75) - np.percentile(world_gdp['GDP_Log'], 25)

#### 3. Which country has the highest GDP? Which country has the lowest? Which countries are outliers? What constitutes the cutoff for an outlier in this case?

In [ ]:
world_gdp[world_gdp['GDP_Log'] == np.max(world_gdp['GDP_Log'])]

In [ ]:
world_gdp[world_gdp['GDP_Log'] == np.min(world_gdp['GDP_Log'])]

In [ ]:
np.percentile(world_gdp['GDP_Log'], 25) - ((np.percentile(world_gdp['GDP_Log'], 75) - np.percentile(world_gdp['GDP_Log'], 25)) * 1.5)

In [ ]:
world_gdp[world_gdp['GDP_Log'] < np.percentile(world_gdp['GDP_Log'], 25) - ((np.percentile(world_gdp['GDP_Log'], 75) - np.percentile(world_gdp['GDP_Log'], 25)) * 1.5)]

#### 4. Now let's join the continents file in and see what we can find.

In [ ]:
continents = pd.read_csv('https://github.com/CSC217/spring_2019/raw/master/week02-descriptive_stats/continents.csv')
world_gdp_new = world_gdp.merge(continents, how='inner', left_on='Country', right_on='Country(or dependent territory)')
world_gdp_new = world_gdp_new[['Country', 'GDP_Log', 'Continent']]
world_gdp_new.head()

#### 5. Plot the swarmplot, boxplot, and ECDF of the gropued GDP Log. Which continent has the highest mean GDP? Which continent has the highest median GDP? Which continent had the highest difference between its median GDP and mean GDP? What do you think accounts for that difference? Which continent has the highest variance? Which continent has the highest range? Do any of the continents have outliers, and if so, which countries are outliers?

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.swarmplot(x='Continent', y='GDP_Log', data=world_gdp_new)
plt.gca().legend(world_gdp_new['Continent'].unique())

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.boxplot(x='Continent', y='GDP_Log', data=world_gdp_new)
plt.gca().legend(world_gdp_new['Continent'].unique())

In [ ]:
#Plot an ECDF
fig = plt.figure(figsize=(10,10))
for continent in world_gdp_new['Continent'].unique():
    n,bins,patches = plt.hist(world_gdp_new[world_gdp_new['Continent'] == continent]['GDP_Log'], cumulative=True, bins=50, density=True, histtype='step');
    #This extra line removes the final line from the graph
    patches[0].set_xy(patches[0].get_xy()[:-1])
plt.gca().legend(world_gdp_new['Continent'].unique(), loc='lower right')

In [ ]:
world_gdp_new.groupby('Continent').mean()

In [ ]:
world_gdp_new.groupby('Continent').median()

In [ ]:
np.abs(world_gdp_new.groupby('Continent').median() - world_gdp_new.groupby('Continent').mean())

In [ ]:
world_gdp_new.groupby('Continent').var()

In [ ]:
world_gdp_new.groupby('Continent')['GDP_Log'].max() - world_gdp_new.groupby('Continent')['GDP_Log'].min()

Europe has the highest mean and median GDP. Oceania had the largest difference in its median and mean because it was right skewed and had an outlier on the high end. Oceania had the highest variance, while North America had the highest range.

In [ ]:
for i in world_gdp_new['Continent'].unique():
    iqr = np.percentile(world_gdp_new[world_gdp_new['Continent'] == i]['GDP_Log'], 75) - np.percentile(world_gdp_new[world_gdp_new['Continent'] == i]['GDP_Log'], 25)
    print('high_outliers', i)
    high_cutoff = np.percentile(world_gdp_new[world_gdp_new['Continent'] == i]['GDP_Log'], 75) + (iqr * 1.5)
    print(world_gdp_new[(world_gdp_new['Continent'] == i) & (world_gdp_new['GDP_Log'] >= high_cutoff)])
    print('low outliers', i)
    low_cutoff = np.percentile(world_gdp_new[world_gdp_new['Continent'] == i]['GDP_Log'], 25) - (iqr * 1.5)
    print(world_gdp_new[(world_gdp_new['Continent'] == i) & (world_gdp_new['GDP_Log'] <= low_cutoff)])

In [ ]:
plt.hist(world_gdp_new[world_gdp_new['Continent'] == 'Oceania']['GDP_Log'])

In [ ]:
plt.hist(world_gdp_new[world_gdp_new['Continent'] == 'North America']['GDP_Log'])

#### 2. Answer Questions 1 through 10 from here below (please copy and paste each question into its own cell with answers below it). I've loaded in the code for you and cleaned it up a bit.

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
df.columns = ['age', 'workingclass', 'fnlwgt', 'education', 'education-num', 'martial-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
df['salary'] = df['salary'].str.strip()
df['native-country'] = df['native-country'].str.strip()

#### 1. How many men and women (sex feature) are represented in this dataset?

In [ ]:
df['sex'].value_counts()

#### 2. What is the average age (age feature) of women?

In [ ]:
df.groupby('sex')['age'].mean()

#### 3. What is the percentage of German citizens (native-country feature)?

In [ ]:
df['native-country'].value_counts(normalize=True)

#### 4-5. What are the mean and standard deviation of age for those who earn more than 50K per year (salary feature) and those who earn less than 50K per year?

In [ ]:
df.groupby('salary')['age'].mean()

In [ ]:
df.groupby('salary')['age'].std()

#### 6. Is it true that people who earn more than 50K have at least high school education? (education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate feature)

In [ ]:
df[df['salary'] == '>50K']['education'].value_counts()

#### 7. Display age statistics for each race (race feature) and each gender (sex feature). Use groupby() and describe(). Find the maximum age of men of Amer-Indian-Eskimo race.

In [ ]:
df.groupby('race')['age'].describe()

#### 8. Among whom is the proportion of those who earn a lot (>50K) greater: married or single men (marital-status feature)? Consider as married those who have a marital-status starting with Married (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.

In [ ]:
df[df['salary'] == '>50K']['martial-status'].value_counts()

#### 9. What is the maximum number of hours a person works per week (hours-per-week feature)? How many people work such a number of hours, and what is the percentage of those who earn a lot (>50K) among them?

In [ ]:
df[df['hours-per-week'] == df['hours-per-week'].max()].shape

In [ ]:
df[df['hours-per-week'] == df['hours-per-week'].max()]['salary'].value_counts(normalize=True)

#### 10. Count the average time of work (hours-per-week) for those who earn a little and a lot (salary) for each country (native-country). What will these be for Japan?

In [ ]:
df[df['native-country'] == 'Japan'].groupby(['native-country', 'salary'])['hours-per-week'].mean()